## 5-9-7 スレッドの優先順位

In [1]:
current = Thread.current
# デフォルトの優先度は 0
current.priority

0

In [2]:
# Thread#priority= で優先度の設定ができる
current.priority = 2
current.priority

2

In [3]:
# 現在のスレッド
current = Thread.current
current.priority = 3

# 新たなスレッドを生成した場合は親のスレッドから優先度を引き継ぐ
Thread.fork do
  puts Thread.current.priority
end

3


#<Thread:0x000055bb568aa3d8@(pry):12 dead>

## 5-9-8 ThreadGroup

In [4]:
group = ThreadGroup.new

p group.list

thread = Thread.fork do
  sleep 1
end

# グループにスレッドを追加
# グループ単位でまとめて何らかの操作を行うことができる
group.add thread

# グループのスレッド一覧
group.list

[]


[#<Thread:0x000055bb568e5ff0@(pry):19 sleep>]

## 5-9-9 Mutex

In [5]:
def countup
  File.open 'counter', File::RDWR | File::CREAT do |f|
    last_count = f.read.to_i
    
    f.rewind
    f.write last_count + 1
  end
end

10.times.map {
  Thread.fork { countup }
}.map(&:join)

# 10回カウントアップしたはずなのに 10 にならないことがある
puts File.read('counter').to_i 

8


In [6]:
File.unlink('counter')
mutex = Mutex.new

10.times.map {
  # Mutex で排他処理を行えば共有データを安全に参照・更新することができる
  Thread.fork do
    mutex.synchronize { countup }
  end
}.map(&:join)

puts File.read('counter').to_i

10


## 5-9-10 デッドロック

In [7]:
m1, m2 = Mutex.new, Mutex.new

t1 = Thread.fork {
  m1.lock
  sleep 1
  m2.lock
}

t2 = Thread.fork {
  m2.lock
  sleep 1
  m1.lock
}

# t1, t2 が互いが保持するロックを待ち続けるので永遠にブロックされる
t1.join
t2.join

Interrupt: 

## 5-9-11 Queue （第2版追加）

In [8]:
queue = Queue.new

#<Thread::Queue:0x000055bb56380bc8>

In [15]:
# Queue を使うと Worker Thread が
workers = 3.times.map {|t|
  Thread.fork {
    while req = queue.deq
      puts "worker#{t} processing..."
      req.call
    end
  }
}

10.times do |t|
  queue.enq -> {
    sleep 1
  }
end

sleep 1 until queue.empty?

p workers.map(&:status)

3.times do |t|
  queue.enq -> { sleep 1}
end

sleep 1 until queue.empty?
p workers.map(&:status)

sleep 1 until queue.empty?

worker1 processing...
worker0 processing...
worker2 processing...
worker1 processing...
worker2 processing...
worker0 processing...
worker1 processing...
worker0 processing...
worker2 processing...
worker2 processing...
["sleep", "sleep", "sleep"]
worker1 processing...
worker0 processing...
worker1 processing...
["sleep", "sleep", "sleep"]


次回は 5-10 Fiber から...